In [18]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [7]:
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [8]:
df = spark.read.csv(
    "clean_file_1.csv",
    sep = ",",
    header = True,
    inferSchema = True,
    timestampFormat = 'yyyy-MM-dd'
)

In [21]:
df.show()

+-----------+-------+-------+-------------+------------------+---------------+-------------+
|Customer_ID|   Name|    Age|  Signup_Date|             Email|Purchase_Amount|      Country|
+-----------+-------+-------+-------------+------------------+---------------+-------------+
|          1|  alice|     25|   2021/01/01| alice@example.com|          10.50|           US|
|          2|    Bob| Thirty|   01-15-2021|   bob@example.com|             20|           us|
|          2|    bob|     30|   01-15-2021|              NULL|         twenty|          USA|
|          3|CHARLIE|     35|March 3, 2021|  charlie@mail.com|           NULL|United States|
|          4|  Diana|     40|         NULL|    diana@mail.com|           35.0|         NULL|
|          5|   NULL|   NULL|   2021-04-01|          eve@mail|             50|           UK|
|       NULL|    Eve|     29|     04/15/21|          eve@mail|          fifty|           uk|
|          7|  Frank|     25|   2021-05-01| frank@example.com|        

In [20]:
# Show Current Schema
df.printSchema()

root
 |-- Customer_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Signup_Date: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Purchase_Amount: string (nullable = true)
 |-- Country: string (nullable = true)



In [19]:
# 1.1 Checking Customer ID
# Change it's type to int
df = df.withColumn("Customer_ID", F.col("Customer_ID").cast(IntegerType()))

In [24]:
# 1.2 Check repeated Primary Numbers
# Get the rows where there is a repeated primary number
df_duplicate = df.groupBy("Customer_ID").agg(F.count("*").alias("count")).where(F.col("count") > 1)

In [28]:
# Check these rows
df_duplicate_row = df.join(df_duplicate, on="Customer_ID", how="inner")
df_duplicate_row.show()

# 

+-----------+----+------+-----------+---------------+---------------+-------+-----+
|Customer_ID|Name|   Age|Signup_Date|          Email|Purchase_Amount|Country|count|
+-----------+----+------+-----------+---------------+---------------+-------+-----+
|          2| Bob|Thirty| 01-15-2021|bob@example.com|             20|     us|    2|
|          2| bob|    30| 01-15-2021|           NULL|         twenty|    USA|    2|
+-----------+----+------+-----------+---------------+---------------+-------+-----+



In [26]:
df_duplicate.show()

+-----------+-----+
|Customer_ID|count|
+-----------+-----+
|       NULL|    2|
|          2|    2|
+-----------+-----+

